In [1]:
import pandas as pd

# Задание 1. Очень усердные ученики

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".
Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин за час.

Дана таблица peas:
* st_id - ученика
* timest - Время решения карточки
* correct - Правильно ли решена горошина?
* subject - Дисциплина, в которой находится горошина

Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов за март 2020 года.

In [5]:
query_1 = """
SELECT
    count(st_id) AS quantity_of_students
FROM
    (
    SELECT 
        st_id,
        count(correct) AS quantity_of_peas
    FROM peas 
    WHERE
        toStartOfMonth(timest) = '2020-03-01'
        AND correct = 1
    GROUP BY st_id
    HAVING quantity_of_peas >= 20
    )
"""

Так в условии задания на LMS изначально были противоречивые формулировки, в которые вносили изменения, и до этого усердным учеником считался пользователь, который хотя бы раз за текущий месяц правильно решил 20 горошин ЗА ЧАС, то вот Postgresql-запрос, который ищет таких учеников за март 2020 года через оконные функции.

In [3]:
query_2 = """
SELECT
    COUNT(st_id) AS quantity_of_students
FROM
    (
    SELECT
        st_id,
        COUNT(correct) OVER w AS quantity_of_peas
    FROM peas
    WHERE
        EXTRACT(MONTH FROM CAST(timest AS date)) = 3.00
        AND EXTRACT(YEAR FROM CAST(timest AS date)) = 2020.00
        AND correct = 1
    WINDOW w AS (
        PARTITION BY st_id
        ORDER BY timest ASC
        RANGE BETWEEN CURRENT ROW AND '1 hour' FOLLOWING
        )
    ) as a
WHERE quantity_of_peas >= 20
"""

# Задание 2.  Оптимизация воронки

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

Даны таблицы: peas, studs, checks.

Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:
* ARPU 
* ARPAU 
* CR в покупку 
* СR активного пользователя в покупку 
* CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике

##### Пояснения к sql-запросу.

Если посмотреть на данные в таблице studs, то мы увидим, что у каждого пользователя несколько меток new и old и больше нет никакой информации, по которой можно было бы нормально сджойнить. Потому что если сджойнить только по st_id все три таблицы, то у нас все перемножится и появится "100500" строк с неверной информацией. У меня сложилось впечатление, что была изначально таблица, которую просто разрезали на 3, так как st_id во всех 3 таблицах отсортирован в одном и том же порядке и количество каждого уникального st_id совпадает. Поэтому за неимением дополнительных данных я добавила в каждую таблицу еще по столбцу с нумерацией и сджойнила по нему и stud_id. НО(!!!) вообще-то в реальности так делать не надо, потому что на больших объемах данных легко допустить ошибку. 

Так как в условии задачи у нас нет понятия "активного пользователя", то предположим, что им будет являться пользователь, решивший хотя бы одну горошину. Им может быть и пользователь, решивший 5, 10, 15 горошин, а может быть пользователь, решивший 20 горошин за месяц. Ну то есть при расчете метрики надо будет просто подставить того самого активного пользователя. 

Расчет метрик:

ARPU (Average Revenue Per User) = доход / количество всех уникальных пользователей

ARPAU (Average Revenue Per Active User)= доход / количество всех уникальных пользователей, решивших хотя бы 1 горошину

CR в покупку = кол-во уникальных пользователей, купивших курсы / кол-во уникальных пользователей, зашедших на сайт 

CR активного пользователя в покупку = кол-во уникальных пользователей, купивших курсы и решивших хотя бы 1 горошину / кол-во всех уникальных пользователей, решивших хотя бы 1 горошину

CR пользователя из активности по математике в покупку курса по математике =  кол-во уникальных пользователей, купивших курсы и решивших хотя бы 1 горошину по математике / кол-во всех уникальных пользователей, решивших хотя бы 1 горошину по математике

In [4]:
query_3 = """
SELECT
    test_grp,
    sum(money) / count(DISTINCT st_id) AS ARPU,
    sum(money) / count(DISTINCT st_id, CASE WHEN correct != 0 THEN 1 ELSE NULL END) AS ARPAU, 
    count(DISTINCT st_id, CASE WHEN money > 0 THEN 1 ELSE NULL END) / count(DISTINCT st_id) AS cr,
    count (DISTINCT st_id, CASE WHEN money>0 AND correct != 0 THEN 1 ELSE NULL END) / count(DISTINCT st_id, CASE WHEN correct != 0 THEN 1 ELSE NULL END) AS cr_AU,
    count(DISTINCT st_id, CASE WHEN money>0 AND subject = 'Math' AND correct != 0 THEN 1 ELSE NULL END) / count(DISTINCT st_id, CASE WHEN correct != 0 AND purchased_subject = 'Math' THEN 1 ELSE NULL END) as cr_math
FROM
    (
    SELECT
        a.st_id AS st_id,
        test_grp,
        timest,
        correct,
        b.subject AS subject,
        sale_time,
        money,
        c.subject AS purchased_subject
    FROM
        (
        SELECT
            st_id,
            test_grp,
            row_number() over(order by st_id) rn
        FROM default.studs
        ) AS a
    JOIN
        (
        SELECT
            st_id,
            toDateTime(timest) AS timest,
            correct,
            subject,
            row_number() over(order by st_id) rn
        FROM default.peas
        ) AS b
        ON a.rn = b.rn AND a.st_id = b.st_id
    JOIN
        (
        SELECT
            st_id,
            toDateTime(sale_time) AS sale_time,
            money,
            subject,
            row_number() over(order by st_id) rn
        FROM default.final_project_check
        ) AS c
        ON a.rn = c.rn AND a.st_id = c.st_id
    )
GROUP BY test_grp
"""